In [1]:
#importing libraries
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#reading xlsx files using pandas
offer_retailer = pd.read_excel(r"offer_retailer.xlsx")
categories = pd.read_excel(r"categories.xlsx")
brand_category = pd.read_excel(r"brand_category.xlsx")

In [3]:
#looking at the top offer_retailer data frame
offer_retailer.head()

,OFFER,RETAILER,BRAND
0,Spend $50 on a Full-Priced new Club Membership,SAMS CLUB,SAMS CLUB
1,"Beyond Meat® Plant-Based products, spend $25",NaN,BEYOND MEAT
2,Good Humor Viennetta Frozen Vanilla Cake,NaN,GOOD HUMOR
3,"Butterball, select varieties, spend $10 at Dil...",DILLONS FOOD STORE,BUTTERBALL
4,"GATORADE® Fast Twitch®, 12-ounce 12 pack, at A...",AMAZON,GATORADE


In [18]:
#looking at the top categories data frame
categories.head()

,CATEGORY_ID,PRODUCT_CATEGORY,IS_CHILD_CATEGORY_TO
0,1f7d2fa7-a1d7-4969-aaf4-1244f232c175,Red Pasta Sauce,Pasta Sauce
1,3e48a9b3-1ab2-4f2d-867d-4a30828afeab,Alfredo & White Pasta Sauce,Pasta Sauce
2,09f3decc-aa93-460d-936c-0ddf06b055a3,Cooking & Baking,Pantry
3,12a89b18-4c01-4048-94b2-0705e0a45f6b,Packaged Seafood,Pantry
4,2caa015a-ca32-4456-a086-621446238783,Feminine Hygeine,Health & Wellness


In [19]:
#looking at the top brand_category data frame
brand_category.head()

,BRAND,BRAND_BELONGS_TO_CATEGORY,RECEIPTS
0,CASEYS GEN STORE,Tobacco Products,2950931
1,CASEYS GEN STORE,Mature,2859240
2,EQUATE,Hair Removal,893268
3,PALMOLIVE,Bath & Body,542562
4,DAWN,Bath & Body,301844


In [20]:
#merging offer_retailer and  brand_category on = "BRAND"
merged_df = pd.merge(brand_category,offer_retailer, on = "BRAND")

In [21]:
#looking at the merged_df
merged_df.head()

,BRAND,BRAND_BELONGS_TO_CATEGORY,RECEIPTS,OFFER,RETAILER
0,DR PEPPER,Carbonated Soft Drinks,77880,"Dr Pepper®, Regular OR Zero Sugar Strawberries...",UNITED SUPERMARKETS
1,DR PEPPER,Cooking & Baking,10983,"Dr Pepper®, Regular OR Zero Sugar Strawberries...",UNITED SUPERMARKETS
2,MOUNTAIN DEW,Carbonated Soft Drinks,68610,"MTN DEW® Kickstart, 16-ounce 12 count, select ...",AMAZON
3,PEPSI,Carbonated Soft Drinks,53604,"PepsiCo® Beverage, 7.5-ounce 10 pack, select v...",AMAZON
4,PEPSI,Carbonated Soft Drinks,53604,"PepsiCo® Variety Pack, select varieties, at Am...",AMAZON


In [23]:
#dropping duplicate values
merged_df.drop_duplicates()

,BRAND,BRAND_BELONGS_TO_CATEGORY,RECEIPTS,OFFER,RETAILER
0,DR PEPPER,Carbonated Soft Drinks,77880,"Dr Pepper®, Regular OR Zero Sugar Strawberries...",UNITED SUPERMARKETS
1,DR PEPPER,Cooking & Baking,10983,"Dr Pepper®, Regular OR Zero Sugar Strawberries...",UNITED SUPERMARKETS
2,MOUNTAIN DEW,Carbonated Soft Drinks,68610,"MTN DEW® Kickstart, 16-ounce 12 count, select ...",AMAZON
3,PEPSI,Carbonated Soft Drinks,53604,"PepsiCo® Beverage, 7.5-ounce 10 pack, select v...",AMAZON
4,PEPSI,Carbonated Soft Drinks,53604,"PepsiCo® Variety Pack, select varieties, at Am...",AMAZON
...,...,...,...,...,...
771,PAVILIONS,Cooking & Baking,15,Spend $155 at Pavilions,PAVILIONS
772,PAVILIONS,Cooking & Baking,15,Shop 2 times at Pavilions,PAVILIONS
773,PAVILIONS,Cooking & Baking,15,Spend $100 at Pavilions,PAVILIONS
774,PAVILIONS,Cooking & Baking,15,Any Pavilions receipt,PAVILIONS


In [24]:
# lemmatizer and stemmer later we save this to pickle files for prediction
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Function to clean and preprocess text
def clean_text(text):
    #substutuing the '-' with " "
    text = re.sub(r'-', ' ', str(text))
    # Removing any special characters and digits from text
    text = re.sub(r'[^a-zA-Z\s]', '', str(text))
    # Tokenize the text
    tokens = word_tokenize(text)
    # Lemmatize each word before lowering
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Remove stopwords
    tokens = [word for word in tokens if word.lower() not in ENGLISH_STOP_WORDS]
    # Stem each word 
    tokens = [stemmer.stem(word) for word in tokens]
    # Convert to lowercase and making them as string
    text = ' '.join(tokens).lower()
    #returning text
    return text

In [25]:
# creating another column with clean text function applied to the "OFFER" column
merged_df['cleaned_offer'] = merged_df['OFFER'].apply(clean_text)
# creating another column with clean text function applied to the "RETAILER" column
merged_df['cleaned_retailer'] = merged_df['RETAILER'].apply(clean_text)
# creating another column with clean text function applied to the "BRAND" column
merged_df['cleaned_brand'] = merged_df['BRAND'].apply(clean_text)
# creating another column with clean text function applied to the "BRAND_BELONGS_TO_CATEGORY" column
merged_df['cleaned_brand_category'] = merged_df['BRAND_BELONGS_TO_CATEGORY'].apply(clean_text)
# creating another column with all cleaned text
merged_df['cleaned_data'] = merged_df['cleaned_offer'] + ' ' + merged_df['cleaned_retailer'] + ' ' + merged_df['cleaned_offer'] + ' '+ merged_df['cleaned_brand_category']

In [26]:
# Create a TF-IDF vectorizer to later save this to pickle file
vectorizer = TfidfVectorizer()
# Fit and transform the processed text
tfidf_matrix = vectorizer.fit_transform(merged_df['cleaned_data'])

In [27]:
# Function to search for offers based on user input
def search_offers(user_input, df, vectorizer, tfidf_matrix):
    # Preprocess user input with same lemmatizer and stemmer
    processed_input = clean_text(user_input)
    # Transform user input using the same vectorizer
    input_vector = vectorizer.transform([processed_input])
    # Calculate cosine similarity between user input and offers
    similarity_scores = cosine_similarity(input_vector, tfidf_matrix).flatten()
    # Add similarity scores to the DataFrame
    df['SIMILARITY_SCORES'] = similarity_scores
    # Sort offers by similarity score
    result_df = df.sort_values(by='SIMILARITY_SCORES', ascending=False)
    return result_df[['OFFER', 'RETAILER', 'BRAND','BRAND_BELONGS_TO_CATEGORY', 'SIMILARITY_SCORES']]

In [28]:
#testing 
user_input_category = "candy"
result_category = search_offers(user_input_category, merged_df, vectorizer, tfidf_matrix)
result_category[:5]

,OFFER,RETAILER,BRAND,BRAND_BELONGS_TO_CATEGORY,SIMILARITY_SCORES
14,"M&M'S® chocolate candies, select varieties",Unknown,M&MS,Candy,0.567937
154,"SNICKERS® chocolate candy bar, select varieties",Unknown,SNICKERS,Candy,0.513689
217,"STARBURST®, select varieties",Unknown,STARBURST,Candy,0.303134
205,"SKITTLES®, select varieties",Unknown,SKITTLES,Candy,0.291478
194,"TWIX®, select varieties",Unknown,TWIX,Candy,0.291478
